In [1]:
import pandas as pd
from plotting import CandlePlot
# show all columns of a data frame on screen
pd.set_option('display.max_columns', None)

In [2]:
df_ma_res = pd.read_pickle("../../assets/data/2022-08-20-11-ma_res.pkl")
df_ma_trades = pd.read_pickle("../../assets/data/2022-08-20-11-ma_trades.pkl")

FileNotFoundError: [Errno 2] No such file or directory: '../../assets/data/2022-08-20-11-ma_res.pkl'

In [ ]:
df_ma_res.info()

In [ ]:
df_ma_res.head()

In [ ]:
# checking what granularitys we have
df_ma_res.granularity.unique() # we have one hour and four hour results

In [ ]:
# taking subset of data for one hour results
df_ma_res_h1 = df_ma_res[df_ma_res.granularity == "H1"].copy()

In [ ]:
df_ma_res_h1.shape

In [ ]:
df_ma_trades.info()

In [ ]:
df_ma_trades.head()

In [ ]:
df_ma_trades.shape

In [ ]:
# checking what granularitys we have
df_ma_trades.granularity.unique() # we have one hour and four hour results

In [ ]:
# taking subset of data for one hour results
df_ma_trades_h1 = df_ma_trades[df_ma_trades.granularity == "H1"].copy()

In [ ]:
df_ma_trades_h1.shape

In [ ]:
# have a look at what crosses where eventually effective
# number of columns on the data frame
df_ma_res_h1.columns

In [ ]:
# group columns by pair, total_gain, num_trades, cross
# copy of data frame of one hourly results
df_cross_summary = df_ma_res_h1[['pair', 'num_trades', 'total_gain', 'cross']].copy()

In [ ]:
df_cross_summary

In [ ]:
df_cross_gr = df_cross_summary.groupby(by="cross").sum()

In [ ]:
df_cross_gr # datadrame where all of the crosses has been grouped

In [ ]:
# fixing nested indexes with as_index
df_cross_gr = df_cross_summary.groupby(by="cross", as_index=False).sum()

In [ ]:
# add sort by 
# modifying the data frame we're actually calling: inplace=True
df_cross_gr.sort_values(by='total_gain', ascending=False, inplace=True)

In [ ]:
df_cross_gr # spread is not taken into account
# taking spread into account would most likely mean that positive looking results are negative
# conclusion: strategy doesn't perform very well

In [ ]:
# what percentage of the crosses we've tested actually gave us a gain?
# pairs we actually tested 
df_ma_res_h1.pair.unique()

In [ ]:
# number of pairs we actually tested
total_pairs = len(df_ma_res_h1.pair.unique())
total_pairs

In [ ]:
# data frame for single cross
temp = df_ma_res_h1[df_ma_res_h1.cross == 'MA_10_MA_40']

In [ ]:
temp

In [ ]:
# how many pairs gave us a positive result?
temp[temp.total_gain > 0] 

In [ ]:
# actual number of columns with a a positive result
temp[temp.total_gain > 0].shape[0]

In [ ]:
# looking at the number of positive results for all the crosses
for c in df_ma_res_h1.cross.unique():
    temp = df_ma_res_h1[df_ma_res_h1.cross == c]
    pg = temp[temp.total_gain > 0].shape[0]
    # crossing, number of positive results, success rate
    print(f"{c} pg: {pg} perc gain: {(pg / total_pairs) * 100:.0f}%")

In [ ]:
# taking a closer look at the MA_10_MA_20 cross
cross = "MA_10_MA_20"
df_MA_10_MA_20 = df_ma_res_h1[df_ma_res_h1.cross == cross].copy()
df_MA_10_MA_20.sort_values(by="total_gain", ascending=False, inplace=True)

In [ ]:
df_MA_10_MA_20

In [ ]:
# next step in analysing: how does gain look over a period of time?
# did we get our gains in just on or two days and did we only have losses after this?
# or did he have wins continously over a wider period of time?
# -> plot the accumulative gain over the period of the backtest
# df_ma_trades_h1 contains all of out trades
# selecting the data we're interested in from the df_ma_trades_h1
df_gains = df_ma_trades_h1[(df_ma_trades_h1.cross == cross) & (df_ma_trades_h1.pair == "GBP_JPY")].copy()

In [ ]:
df_gains # all gains for the GBP_JPY pair for the MA_10_MA_20 cross

In [ ]:
cp = CandlePlot(df_gains, candles=False)
cp.show_plot(line_traces=['GAIN_C'])

In [ ]:
# similar analysis for the EUR_AUD pair
df_gains_EUR_AUD = df_ma_trades_h1[(df_ma_trades_h1.cross == cross) & (df_ma_trades_h1.pair == "EUR_AUD")].copy()

In [ ]:
df_gains_EUR_AUD

In [ ]:
cp = CandlePlot(df_gains_EUR_AUD, candles=False)
cp.show_plot(line_traces=['GAIN_C'])

In [ ]:
# looping through all of the uniquen pairs of the df_MA_10_MA_20 where the total gain is greater than zero
for curr in df_MA_10_MA_20[df_MA_10_MA_20.total_gain > 0].pair.unique():
    temp_gains = df_ma_trades_h1[(df_ma_trades_h1.cross == cross) & 
        (df_ma_trades_h1.pair == curr)].copy()
    print(curr, int(temp_gains.GAIN.sum()))
    cp = CandlePlot(temp_gains, candles=False)
    cp.show_plot(line_traces=['GAIN_C'])